![vki_logo](https://www.skywin.be/sites/default/files/logo-membres/vki_logo_blue_rectangular.jpg)
### MODULO: A package for Multiscale Proper Orthogonal Decomposition
#### Tutorial 2:  DFT, POD, SPODs and mPOD of an impinging gas jet 

This second tutorial considers a dataset which is dynamically much richer than the previous. This time 3 POD modes cannot the essence of what is happening. This data is the TR-PIV of an impinging gas jet and was extensively analyzed in previous tutorials on MODULO.
It was also discussed in the [Mendez et al, 2018](
https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/article/abs/multiscale-proper-orthogonal-decomposition-of-complex-fluid-flows/D078BD2873B1C30B6DD9016E30B62DA8 ) and Chapter 8 of the book [Mendez et al, 2022](https://www.cambridge.org/core/books/datadriven-fluid-mechanics/0327A1A43F7C67EE88BB13743FD9DC8D).

We refer you to those work for details on the experimental set up and the flow conditions.


#### Load packages and prepare plot customization (Optional)

In [ ]:
import numpy as np # we use this to manipulate data 
import matplotlib.pyplot as plt # this is for plotting
import os  # this is to create/rename/delete folders
from modulo_vki import ModuloVKI # this is to create modulo objects

# these are some utility functions 
from modulo_vki.utils.others import Plot_Field_TEXT_JET, Plot_Field_JET # plotting
from modulo_vki.utils.others import Animation_JET # for animations 
from modulo_vki.utils.read_db import ReadData # to read the data

# This is for plot customization
fontsize = 16
plt.rc('text', usetex=True)      
plt.rc('font', family='serif')
plt.rcParams['xtick.labelsize'] = fontsize
plt.rcParams['ytick.labelsize'] = fontsize
plt.rcParams['axes.labelsize'] = fontsize
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams['font.size'] = fontsize

FOLDER='Tutorial_2_JET_PIV'


#### Download the dataset and store it into a local folder+ prepare folder for results

###### Load one snapshot and plot it 
We use the functions Plot_Field_TEXT_JET to extract all the data concerning the on snapshot. In particular, we us it to extract the grid information. Note that the grid is here saved in each of the dat files, even if this is identical for all of them. This is very inefficient, but that's how the output was produced from our old PIV code. In what follows we generate the time and space grid and plot the velocity field of snapshot 10.

In [ ]:
# Construct Time discretization
n_t=2000; Fs=2000; dt=1/Fs 
t=np.linspace(0,dt*(n_t-1),n_t) # prepare the time axis# 

# Read file number 10 (Check the string construction)
SNAP=10
Name=FOLDER+os.sep+'data/Res%05d'%SNAP+'.dat' # Check it out: print(Name)
n_s, Xg, Yg, Vxg, Vyg, X_S,Y_S=Plot_Field_TEXT_JET(Name); plt.close() 
# Shape of the grid
n_y,n_x=np.shape(Xg)

# Plot the vector field
fig, ax = plt.subplots(figsize=(5, 3)) # This creates the figure
Magn=np.sqrt(Vxg**2+Vyg**2); 
CL=plt.contourf(Xg,Yg,Magn,levels=np.arange(0,9,1))
STEPx=2; STEPy=2; 
plt.quiver(Xg[::STEPx,::STEPy],Yg[::STEPx,::STEPy],\
           Vxg[::STEPx,::STEPy],Vyg[::STEPx,::STEPy],color='k',scale=100) # Create a quiver (arrows) plot
    
plt.rc('text', usetex=True)      # This is Miguel's customization
plt.rc('font', family='serif')
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
fig.colorbar(CL,pad=0.05,fraction=0.025)
ax.set_aspect('equal') # Set equal aspect ratio
ax.set_xlabel('$x[mm]$',fontsize=13)
ax.set_ylabel('$y[mm]$',fontsize=13)
ax.set_title('Tutorial 2: Impinging Jet',fontsize=16)
ax.set_xticks(np.arange(0,40,10))
ax.set_yticks(np.arange(10,30,5))
ax.set_xlim([0,35])
ax.set_ylim(10,29)
ax.invert_yaxis() # Invert Axis for plotting purpose
plt.tight_layout()
Name=FOLDER+os.sep+'Snapshot_JET_'+str(SNAP)+'.png'
plt.savefig(Name, dpi=200) 
plt.show()




##### Step 1 Load the data and create snapshot matrix D


In [ ]:
# --- Component fields (N=2 for 2D velocity fields, N=1 for pressure fields)
N = 2 
# --- Number of mesh points
N_S = 6840
# --- Header (H) and footer (F) to be skipped during acquisition
H = 1; F = 0
# --- Read one sample snapshot (to get N_S)
Name = FOLDER+"/data/Res00001.dat"
Dat = np.genfromtxt(Name, skip_header=H, skip_footer=F)

D = ReadData._data_processing(D=None,
                              FOLDER_OUT='./',
                              FOLDER_IN=FOLDER+'/data/', 
                              filename='Res%05d', 
                              h=H,f=F,c=2,
                              N=2, N_S=2*Dat.shape[0],N_T=n_t)


##### Plot one snapshot  and the animation of the velocity field

To plot an animation of the velocity field, we use the function Animation_JET. This requires in input the snapshot matrix previously created


In [ ]:
Name_GIF=FOLDER+os.sep+'Velocity_Field.gif'
Mex=Animation_JET(Name_GIF,D,X_S,Y_S,500,600,1)

## Step 2: Perform POD analysis


With the POD, we put no constraints on the frequency content of each mode and instead seek to find the optimal basis, i.e., the one able to represent the data with the least amount of modes.
We compute the POD with the snapshot method, using scipy's eigsh for the diagonalization. However, we also initialize the modulo object with an svd solver, which we will use for other decompositions in the following. 

After having studied tutorial 1, the code for the decomposition and the plot that follows should be fairly easy to follow.


In [ ]:
# --- Remove the mean from this dataset (stationary flow )!
D,D_MEAN=ReadData._data_processing(D,MR=True)
# We create a matrix of mean flow (used to sum again for the videos):
D_MEAN_mat=np.array([D_MEAN, ] * n_t).transpose()    

# --- Initialize MODULO object
m = ModuloVKI(data=D)
# Compute the DFT
Sorted_Freqs, Phi_F, Sorted_Sigmas = m.compute_DFT(Fs)

# Shape of the grid
nxny=m.D.shape[0]//2; 


In [ ]:
# The POD provides the best decomposition convergence.
# Here is how to perform it:
# --- Initialize MODULO object
m2 = ModuloVKI(data=m.D,n_Modes=50)
# --- Check for D
Phi_P, Psi_P, Sigma_P = m2.compute_POD_svd() # POD via svd

FOLDER_POD_RESULTS=FOLDER+os.sep+'POD_Results_Jet'
if not os.path.exists(FOLDER_POD_RESULTS):
    os.mkdir(FOLDER_POD_RESULTS)

# Plot the decomposition convergence
fig, ax = plt.subplots(figsize=(6, 3)) # This creates the figure
plt.plot(Sigma_P/np.max(Sigma_P),'ko:')
# ax.set_yscale('log'); ax.set_xscale('log')
plt.xlabel('$r$',fontsize=18)
plt.ylabel('$\sigma_{\mathcal{P}r}/(\sigma_{\mathcal{P}1})$',fontsize=18)
plt.tight_layout(pad=0.6, w_pad=0.3, h_pad=0.8)
Name=FOLDER_POD_RESULTS+os.sep+'POD_R_Impinging_JET3.png'
plt.savefig(Name, dpi=200) 
plt.show()

# Plot the leading POD modes and their spectra:
plt.ion()
# Show modes
for j in range(1,10):
 plt.close(fig='all') 
 fig, ax3= plt.subplots(figsize=(5,6))   
 ax=plt.subplot(2,1,1)
 plt.rc('text', usetex=True)    
 plt.rc('font', family='serif')
 plt.rc('xtick',labelsize=12)
 plt.rc('ytick',labelsize=12)
 V_X=Phi_P[0:nxny,j-1]
 V_Y=Phi_P[nxny::,j-1]
 Plot_Field_JET(X_S,Y_S,V_X,V_Y,True,2,1)
 #plt.quiver(X_S,Y_S,V_X,V_Y)
 ax.set_aspect('equal') # Set equal aspect ratio
 ax.set_xticks(np.arange(0,40,10))
 ax.set_yticks(np.arange(10,30,5))
 ax.set_xlim([0,35])
 ax.set_ylim([10,29])
 ax.set_xlabel('$x[mm]$',fontsize=13)
 ax.set_ylabel('$y[mm]$',fontsize=13)
 ax.invert_yaxis() # Invert Axis for plotting purpose
 String_y='$\phi_{\mathcal{S}'+str(j)+'}$'
 plt.title(String_y,fontsize=18)
 plt.tight_layout(pad=1, w_pad=0.5, h_pad=1.0)
 
 ax=plt.subplot(2,1,2)
 Signal=Psi_P[:,j-1]
 s_h=np.abs((np.fft.fft(Signal-Signal.mean())))
 Freqs=np.fft.fftfreq(int(n_t))*Fs
 plt.plot(Freqs*(4/1000)/6.5,s_h,'-',linewidth=1.5)
 plt.xlim(0,0.38)    
 plt.xlabel('$St[-]$',fontsize=18)
 String_y='$\widehat{\psi}_{\mathcal{S}'+str(j)+'}$'
 plt.ylabel(String_y,fontsize=18)
 plt.tight_layout(pad=1, w_pad=0.5, h_pad=1.0)
 Name=FOLDER_POD_RESULTS+os.sep+'POD_s_Mode50_'+str(j)+'.png'
 print(Name+' Saved')
 plt.savefig(Name, dpi=300) 


Study the results in the folder. The convergence is much stronger than in the DFT. However the modes have a broad range of frequencies. This is the spectral mixing described in [Mendez et al, 2018](
https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/article/abs/multiscale-proper-orthogonal-decomposition-of-complex-fluid-flows/D078BD2873B1C30B6DD9016E30B62DA8 ) and Chapter 8 of the book [Mendez et al, 2022](https://www.cambridge.org/core/books/datadriven-fluid-mechanics/0327A1A43F7C67EE88BB13743FD9DC8D). Here we plot an animation of the flow field using the leading 50 POD modes. You should compare this with the one of the full dataset 

In [ ]:
# Here is the approximation with the leading 50 POD modes
D_P=np.real(np.linalg.multi_dot([Phi_P,np.diag(Sigma_P),Psi_P.T]) )
Error=np.linalg.norm(m.D-D_P)/np.linalg.norm(m.D)

print('Convergence Error: E_C='+"{:.2f}".format(Error*100)+' %')

Name_GIF=FOLDER_POD_RESULTS+os.sep+'POD_Approximation_R50.gif'
plt.ioff()
Mex=Animation_JET(Name_GIF,D_P+D_MEAN_mat,X_S,Y_S,500,600,1)


In [ ]:
#%% Plot the trajectory

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')

#ax.dist = 22
ax.scatter(Psi_P[1:13000:1, 0],
           Psi_P[1:13000:1, 1],
           Psi_P[1:13000:1, 2])


plt.rc('text', usetex=True)      
plt.rc('font', family='serif')
plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)


ax.set_xlabel('$\psi_{\mathcal{P}1}$',fontsize=16)
ax.set_ylabel('$\psi_{\mathcal{P}2}$',fontsize=16)
ax.set_zlabel('$\psi_{\mathcal{P}3}$',fontsize=16)
plt.tight_layout()
Name='3D_POD_ImpJet_POD50.pdf'
plt.savefig(Name, dpi=300) 
plt.show()

The extreme convergence comes at the prices of spectral mixing. 
Modes are characterized by a large range of frequencies and thus their spatial structures cannot be associated to specific ranges of frequencies.


This is an hybrid between DFT and POD. We filter the correlation matrix
to make it more circulant. At the limit of perfectly circulant matrix,
the POD is a DFT.The filtering is carried out along diagonals.


In [ ]:
FOLDER_SPOD_RESULTS=FOLDER+os.sep+'SPOD_s_Results_Jet'
if not os.path.exists(FOLDER_SPOD_RESULTS):
    os.mkdir(FOLDER_SPOD_RESULTS)


# Initialize a 'MODULO Object'
m = ModuloVKI(data=m.D)
# Prepare (partition) the dataset
# Compute the POD
Phi_S, Psi_S, Sigma_S = m.compute_SPOD_s(Fs,N_O=100,
                                               f_c=0.01,
                                               n_Modes=25,
                                               SAVE_SPOD=True)
 
# The rest of the plotting is IDENTICAL to the POD part

fig, ax = plt.subplots(figsize=(6, 3)) # This creates the figure
plt.plot(Sigma_S/np.max(Sigma_S),'ko:')
# ax.set_yscale('log'); ax.set_xscale('log')
plt.xlabel('$r$',fontsize=18)
plt.ylabel('$\sigma_{\mathcal{S}r}/(\sigma_{\mathcal{S}1})$',fontsize=18)
plt.tight_layout(pad=0.6, w_pad=0.3, h_pad=0.8)
Name=FOLDER_SPOD_RESULTS+os.sep+'SPOD_R_Impinging_JET.png'
plt.savefig(Name, dpi=200) 
plt.show()

# Plot the leading SPOD modes and their spectra:
    
# Show modes
for j in range(1,10):
 plt.close(fig='all') 
 fig, ax3= plt.subplots(figsize=(5,6))   
 ax=plt.subplot(2,1,1)
 plt.rc('text', usetex=True)    
 plt.rc('font', family='serif')
 plt.rc('xtick',labelsize=12)
 plt.rc('ytick',labelsize=12)
 V_X=Phi_S[0:nxny,j-1]
 V_Y=Phi_S[nxny::,j-1]
 Plot_Field_JET(X_S,Y_S,V_X,V_Y,True,2,1)
 #plt.quiver(X_S,Y_S,V_X,V_Y)
 ax.set_aspect('equal') # Set equal aspect ratio
 ax.set_xticks(np.arange(0,40,10))
 ax.set_yticks(np.arange(10,30,5))
 ax.set_xlim([0,35])
 ax.set_ylim([10,29])
 ax.set_xlabel('$x[mm]$',fontsize=13)
 ax.set_ylabel('$y[mm]$',fontsize=13)
 ax.invert_yaxis() # Invert Axis for plotting purpose
 String_y='$\phi_{\mathcal{S}'+str(j)+'}$'
 plt.title(String_y,fontsize=18)
 plt.tight_layout(pad=1, w_pad=0.5, h_pad=1.0)
 
 ax=plt.subplot(2,1,2)
 Signal=Psi_S[:,j-1]
 s_h=np.abs((np.fft.fft(Signal-Signal.mean())))
 Freqs=np.fft.fftfreq(int(n_t))*Fs
 plt.plot(Freqs*(4/1000)/6.5,s_h,'-',linewidth=1.5)
 plt.xlim(0,0.38)    
 plt.xlabel('$St[-]$',fontsize=18)
 String_y='$\widehat{\psi}_{\mathcal{S}'+str(j)+'}$'
 plt.ylabel(String_y,fontsize=18)
 plt.tight_layout(pad=1, w_pad=0.5, h_pad=1.0)
 Name=FOLDER_SPOD_RESULTS+os.sep+'SPOD_s_Mode_'+str(j)+'.png'
 print(Name+' Saved')
 plt.savefig(Name, dpi=300) 


### SPOD_s Conclusions 

We can re-build the data more easily. We have significantly reduced the spectral bandwidth of the modes, at the cost of a minor loss in the convergence (can you appreciate that ?). On the other hand we do not have full control on the frequency content of each mode.


## Step 5: Perform mPOD analysis

Here we go for the mPOD by [Mendez et al 2019](https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/article/abs/multiscale-proper-orthogonal-decomposition-of-complex-fluid-flows/D078BD2873B1C30B6DD9016E30B62DA8). Like Sirovinch's SPOD this decomposition modifies the standard POD by acting on the matrix K . However, the mPOD does not just filter it: it decomposes into chunks containing certain portions of the spectra and diagonalizes each of these independently. The resulting modes are optimal within the user-provided frequency repartition.

In [ ]:
FOLDER_MPOD_RESULTS=FOLDER+os.sep+'mPOD_Results_Jet'
if not os.path.exists(FOLDER_MPOD_RESULTS):
    os.mkdir(FOLDER_MPOD_RESULTS)

# We here perform the mPOD as done in the previous tutorials.
# This is mostly a copy paste from those, but we include it for completenetss
Keep = np.array([1, 1, 1, 1])
Nf = np.array([201, 201, 201, 201])
# --- Test Case Data:
# + Stand off distance nozzle to plate
H = 4 / 1000  
# + Mean velocity of the jet at the outlet
U0 = 6.5  
# + Input frequency splitting vector in dimensionless form (Strohual Number)
ST_V = np.array([0.1, 0.2, 0.25, 0.4])  
# + Frequency Splitting Vector in Hz
F_V = ST_V * U0 / H
# + Size of the extension for the BC (Check Docs)
Ex = 203  # This must be at least as Nf.
dt = 1/2000; boundaries = 'reflective'; MODE = 'reduced'
# Here 's the mPOD
Phi_M, Psi_M, Sigmas_M = m.compute_mPOD(Nf, Ex, F_V, Keep, 20 ,boundaries, MODE, dt, False)


# The rest of the plotting is IDENTICAL to the POD part
fig, ax = plt.subplots(figsize=(6, 3)) # This creates the figure
plt.plot(Sigmas_M/np.max(Sigmas_M),'ko:')
# ax.set_yscale('log'); ax.set_xscale('log')
plt.xlabel('$r$',fontsize=18)
plt.ylabel('$\sigma_{\mathcal{M}r}/(\sigma_{\mathcal{M}1})$',fontsize=18)
plt.tight_layout(pad=0.6, w_pad=0.3, h_pad=0.8)
Name=FOLDER_MPOD_RESULTS+os.sep+'mPOD_R_Impinging_JET.png'
plt.savefig(Name, dpi=200) 
plt.show()

# Plot the leading mPOD modes and their spectra:
    
# Show modes
for j in range(1,5):
 plt.close(fig='all') 
 fig, ax3= plt.subplots(figsize=(5,6))   
 ax=plt.subplot(2,1,1)
 plt.rc('text', usetex=True)    
 plt.rc('font', family='serif')
 plt.rc('xtick',labelsize=12)
 plt.rc('ytick',labelsize=12)
 V_X=Phi_M[0:nxny,j-1]
 V_Y=Phi_M[nxny::,j-1]
 Plot_Field_JET(X_S,Y_S,V_X,V_Y,True,2,1)
 #plt.quiver(X_S,Y_S,V_X,V_Y)
 ax.set_aspect('equal') # Set equal aspect ratio
 ax.set_xticks(np.arange(0,40,10))
 ax.set_yticks(np.arange(10,30,5))
 ax.set_xlim([0,35])
 ax.set_ylim([10,29])
 ax.set_xlabel('$x[mm]$',fontsize=13)
 ax.set_ylabel('$y[mm]$',fontsize=13)
 ax.invert_yaxis() # Invert Axis for plotting purpose
 String_y='$\phi_{\mathcal{M}'+str(j)+'}$'
 plt.title(String_y,fontsize=18)
 plt.tight_layout(pad=1, w_pad=0.5, h_pad=1.0)
 
 ax=plt.subplot(2,1,2)
 Signal=Psi_M[:,j-1]
 s_h=np.abs((np.fft.fft(Signal-Signal.mean())))
 Freqs=np.fft.fftfreq(int(n_t))*Fs
 plt.plot(Freqs*(4/1000)/6.5,s_h,'-',linewidth=1.5)
 plt.xlim(0,0.38)    
 plt.xlabel('$St[-]$',fontsize=18)
 String_y='$\widehat{\psi}_{\mathcal{M}'+str(j)+'}$'
 plt.ylabel(String_y,fontsize=18)
 plt.tight_layout(pad=1, w_pad=0.5, h_pad=1.0)
 Name=FOLDER_MPOD_RESULTS+os.sep+'mPOD_s_Mode_'+str(j)+'.png'
 print(Name+' Saved')
 plt.savefig(Name, dpi=300) 



### mPOD Conclusions 
The convergence is still great and we have clear spectral separation between the modes ( check these!)


In [ ]:
# Here is the approximation with the leading 10 mPOD modes
D_P=np.real(np.linalg.multi_dot([Phi_M,np.diag(Sigmas_M),Psi_M.T]) )
Error=np.linalg.norm(m.D-D_P)/np.linalg.norm(m.D)

print('Convergence Error: E_C='+"{:.2f}".format(Error*100)+' %')

Name_GIF=FOLDER_MPOD_RESULTS+os.sep+'mPOD_Approximation.gif'
Mex=Animation_JET(Name_GIF,D_P+D_MEAN_mat,X_S,Y_S,500,600,1)


In [ ]:
#%% Plot the trajectory

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')

#ax.dist = 22
ax.scatter(Psi_M[1:13000:1, 0],
           Psi_M[1:13000:1, 1],
           Psi_M[1:13000:1, 2])


plt.rc('text', usetex=True)      
plt.rc('font', family='serif')
plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)


ax.set_xlabel('$\psi_{\mathcal{P}1}$',fontsize=16)
ax.set_ylabel('$\psi_{\mathcal{P}2}$',fontsize=16)
ax.set_zlabel('$\psi_{\mathcal{P}3}$',fontsize=16)
plt.tight_layout()
Name=FOLDER_MPOD_RESULTS+os.sep+'3D_POD_ImpJet_mPOD.pdf'
plt.savefig(Name, dpi=300) 
plt.show()